## Prompt Injection

### Mock data maken

In [58]:
import pandas as pd
from faker import Faker
import random
import json
from huggingface_hub import login
from dotenv import load_dotenv
import os 
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from datasets import load_dataset
from ollama import chat
from ollama import ChatResponse

In [7]:
fake = Faker()
Faker.seed(42)
random.seed(42)

In [9]:
def generate_patient_data(num_patients=100):
    patient_data = []
    for _ in range(num_patients):
        patient = {
            "PatientName": fake.name(),
            "Age": random.randint(1, 100),
            "Gender": random.choice(["Male", "Female", "Other"]),
            "Diagnosis": random.choice(["Hypertension", "Diabetes", "Asthma", "Heart Failure", "Depression"]),
            "Medications": random.choice(["Metformin", "Paracetamol", "Lisinopril", "Salbutamol", "Sertraline"]),
            "AdmissionDate": fake.date_this_year().strftime("%Y-%m-%d"),
            "DischargeDate": fake.date_this_year().strftime("%Y-%m-%d"),
            "Department": random.choice(["Cardiology", "Psychiatry", "Endocrinology", "Pulmonology"]),
            "BloodPressure": f"{random.randint(90, 180)}/{random.randint(60, 120)}",
            "Lifestyle": {
                "Smoking": random.choice(["Yes", "No"]),
                "Alcohol": random.choice(["Yes", "No"]),
                "PhysicalActivity": random.choice(["Low", "Moderate", "High"])
            },
            "Symptoms": random.choice(["Chest pain", "Shortness of breath", "Headache", "Fatigue", "Nausea"])
        }
        patient_data.append(patient)
    return patient_data

In [10]:
patient_df = generate_patient_data(100)

### Mock data omzetten naar ander format

Llama werkt het best met een dataset dat bestaat uit teksten in plaats van rijen en kolommen.

In [11]:
def generate_instructions():
    return random.choice([
        "Summarize the patient's medical profile.",
        "What is the diagnosis for this patient?",
        "What medications are prescribed to the patient?",
        "Suggest lifestyle changes for the patient based on their medical history.",
        "Assess the patient's risk level based on their vitals and medical history.",
        "What symptoms are reported by the patient?",
        "Explain the patient's condition in simple terms for their family.",
        "What department is the patient admitted to?",
        "When was the patient last admitted to the hospital?"
    ])

In [12]:
def convert_to_llama_format(patient_data):
    llama_data = []
    for patient in patient_data:
        instruction = generate_instructions()
        if "Summarize" in instruction:
            input_data = (
                f"Name: {patient['PatientName']}, Age: {patient['Age']}, Gender: {patient['Gender']}, "
                f"Blood Pressure: {patient['BloodPressure']}, Medications: {patient['Medications']}, Department: {patient['Department']}"
            )
            output_data = (
                f"{patient['PatientName']} is a {patient['Age']}-year-old {patient['Gender'].lower()} "
                f"treated in the {patient['Department']} department. "
                f"The patient has a blood pressure reading of {patient['BloodPressure']} and is prescribed {patient['Medications']}."
            )
        elif "What is the diagnosis" in instruction:
            input_data = (
                f"Name: {patient['PatientName']}, Symptoms: {patient['Symptoms']}, Blood Pressure: {patient['BloodPressure']}, Department: {patient['Department']}"
            )
            output_data = f"The patient is diagnosed with {patient['Diagnosis']}."
        elif "What medications" in instruction:
            input_data = (
                f"Name: {patient['PatientName']}, Age: {patient['Age']}, Medications: {patient['Medications']}"
            )
            output_data = f"The patient is prescribed {patient['Medications']}."
        elif "Suggest lifestyle changes" in instruction:
            input_data = (
                f"Name: {patient['PatientName']}, Age: {patient['Age']}, Lifestyle: Smoking: {patient['Lifestyle']['Smoking']}, "
                f"Alcohol: {patient['Lifestyle']['Alcohol']}, Physical Activity: {patient['Lifestyle']['PhysicalActivity']}"
            )
            output_data = (
                f"The patient should quit smoking, reduce alcohol consumption, and increase physical activity to a moderate level."
                if patient['Lifestyle']['Smoking'] == "Yes" or patient['Lifestyle']['Alcohol'] == "Yes" else
                "The patient's lifestyle is healthy, no significant changes needed."
            )
        elif "Assess the patient's risk level" in instruction:
            input_data = (
                f"Name: {patient['PatientName']}, Age: {patient['Age']}, Blood Pressure: {patient['BloodPressure']}, "
                f"Medical History: {patient['Diagnosis']}"
            )
            output_data = (
                f"The patient has a high risk of complications due to {patient['Diagnosis'].lower()} and elevated blood pressure."
                if "Hypertension" in patient['Diagnosis'] else
                f"The patient's risk level is moderate based on their medical history."
            )
        elif "What symptoms" in instruction:
            input_data = (
                f"Name: {patient['PatientName']}, Age: {patient['Age']}, Symptoms: {patient['Symptoms']}"
            )
            output_data = f"The patient reports {patient['Symptoms']}."
        elif "Explain the patient's condition" in instruction:
            input_data = (
                f"Name: {patient['PatientName']}, Age: {patient['Age']}, Diagnosis: {patient['Diagnosis']}"
            )
            output_data = (
                f"{patient['PatientName']} has been diagnosed with {patient['Diagnosis'].lower()}, which requires ongoing management and treatment."
            )
        elif "What department" in instruction:
            input_data = (
                f"Name: {patient['PatientName']}, Department: {patient['Department']}"
            )
            output_data = f"The patient is admitted to the {patient['Department']} department."
        elif "When was the patient last admitted" in instruction:
            input_data = (
                f"Name: {patient['PatientName']}, Admission Date: {patient['AdmissionDate']}"
            )
            output_data = f"The patient was last admitted on {patient['AdmissionDate']}."

        example = {
            "instruction": instruction,
            "input": input_data,
            "output": output_data
        }
        llama_data.append(example)
    return llama_data

In [14]:
llama_data = convert_to_llama_format(patient_df)

In [15]:
output_file = "mock_patient_data.jsonl"
with open(output_file, "w") as f:
    for entry in llama_data:
        f.write(json.dumps(entry) + "\n")

print(f"Dataset opgeslagen in {output_file}")

Dataset opgeslagen in mock_patient_data.jsonl


# LLaMA toepassen

## Ollama

Ollama biedt verschillende modellen aan, deze kunnen geïnstalleerd worden en lokaal gedraaid worden.


In [35]:
# Lees het bestand in en laad de data in een lijst
llama_data = []
with open("mock_patient_data.jsonl", "r") as f:
    for line in f:
        llama_data.append(json.loads(line.strip()))

print(f"Loaded {len(llama_data)} records from the JSONL file.")


Loaded 100 records from the JSONL file.


In [59]:
response: ChatResponse = chat(model='llama3.2', messages=[
  {
    'role': 'user',
    'content': 'Why is the sky blue?',
  },
])
print(response.message.content)

The sky appears blue because of a phenomenon called Rayleigh scattering, named after the British physicist Lord Rayleigh, who first described it in the late 19th century.

Here's what happens:

1. **Sunlight enters Earth's atmosphere**: When sunlight enters our atmosphere, it encounters tiny molecules of gases such as nitrogen (N2) and oxygen (O2).
2. **Scattering occurs**: These gas molecules scatter the light in all directions, but they scatter shorter (blue) wavelengths more than longer (red) wavelengths.
3. **Blue light is scattered more**: The blue light is scattered by the smaller gas molecules, which are much smaller than the wavelength of light. This scattering effect is more pronounced for shorter wavelengths because they have a larger cross-sectional area relative to their length.
4. **Our eyes see the scattered blue light**: When we look at the sky, our eyes see the blue light that has been scattered by the gas molecules in all directions. Since there are more gas molecules 

In [60]:
stream = chat(
    model='llama3.2',
    messages=[{'role': 'user', 'content': 'Why is the sky blue?'}],
    stream=True,
)

for chunk in stream:
  print(chunk['message']['content'], end='', flush=True)

The sky appears blue because of a phenomenon called Rayleigh scattering, named after the British physicist Lord Rayleigh, who first described it in the late 19th century.

Here's what happens:

1. When sunlight enters Earth's atmosphere, it encounters tiny molecules of gases such as nitrogen (N2) and oxygen (O2).
2. These molecules are much smaller than the wavelength of light, which means they scatter the shorter (blue) wavelengths more than the longer (red) wavelengths.
3. As a result, the blue light is scattered in all directions by the tiny molecules, while the red light continues to travel in a straight line.
4. Our eyes see the blue light that's been scattered in all directions, which is why the sky appears blue during the daytime.

The color of the sky can also change depending on several factors, such as:

* Time of day: During sunrise and sunset, the light travels through more of the atmosphere, scattering off more particles and changing the apparent color of the sky.
* Atmosp

In [61]:
import ollama

modelfile='''
FROM llama3.2
SYSTEM You are mario from super mario bros.
'''

ollama.create(model='example', modelfile=modelfile)

ShowResponse(modified_at=datetime.datetime(2024, 12, 9, 13, 59, 15, 669547, tzinfo=TzInfo(+01:00)), template='<|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\n\n{{ if .System }}{{ .System }}\n{{- end }}\n{{- if .Tools }}When you receive a tool call response, use the output to format an answer to the orginal user question.\n\nYou are a helpful assistant with tool calling capabilities.\n{{- end }}<|eot_id|>\n{{- range $i, $_ := .Messages }}\n{{- $last := eq (len (slice $.Messages $i)) 1 }}\n{{- if eq .Role "user" }}<|start_header_id|>user<|end_header_id|>\n{{- if and $.Tools $last }}\n\nGiven the following functions, please respond with a JSON for a function call with its proper arguments that best answers the given prompt.\n\nRespond in the format {"name": function name, "parameters": dictionary of argument name and its value}. Do not use variables.\n\n{{ range $.Tools }}\n{{- . }}\n{{ end }}\n{{ .Content }}<|eot_id|>\n{{- else }}\n\n{{ .Content }}<|e

In [18]:
import subprocess

ollama_path = r"C:\Users\rensv\AppData\Local\Programs\Ollama\ollama.exe"

prompt = "What is the weather like today in The Netherlands?"

# Voer het model uit via Ollama met de prompt
result = subprocess.run(
    [ollama_path, "run", "llama3.2"],
    input=prompt.encode("utf-8"),
    stdout=subprocess.PIPE
)

# Print het antwoord van het model
print(result.stdout.decode("utf-8"))


I'm not able to provide real-time weather information. However, I can suggest some ways for you to find out the current weather conditions in The Netherlands:

1. Check online weather websites: You can check websites like AccuWeather, Weather.com, or the Royal Meteorological Institute of the Netherlands (KNMI) for up-to-date weather forecasts and conditions.
2. Use a mobile app: Download a weather app on your smartphone, such as Dark Sky or Weather Underground, to get current weather conditions and forecasts for The Netherlands.
3. Check social media: Follow local news outlets or tourism boards on social media platforms like Twitter or Instagram to get updates on the current weather in The Netherlands.

Please note that I'm a large language model, I don't have real-time access to current weather information. However, I can provide general information about the typical climate and weather patterns in The Netherlands if you're planning a trip there.




In [50]:
# Define preprompt and prompt for LLaMA
llama_preprompt = f"""
You have access to medical records of patients. Here is the dataset:

{json.dumps(llama_data[:5])}

You can now answer questions related to these patients' data, such as their diagnosis, medications, medical history, symptoms, and more.
"""

# Example query for LLaMA
user_query = "Can you name all the patients?"

full_prompt = llama_preprompt + "\n" + user_query

# Run the LLaMA model with the combined prompt
result = subprocess.run(
    [ollama_path, "run", "llama3.2"],
    input=full_prompt.encode("utf-8"),
    stdout=subprocess.PIPE
)

# Print the result
print(result.stdout.decode("utf-8"))

The names of the patients mentioned in the dataset are:

1. Allison Hill
2. Megan Mcclain
3. Allen Robinson
4. Cristian Santos
5. Kevin Pacheco




In [51]:
user_query = "Can you tell me what medication Allison Hill uses?"

full_prompt = llama_preprompt + "\n" + user_query

# Run the LLaMA model with the combined prompt
result = subprocess.run(
    [ollama_path, "run", "llama3.2"],
    input=full_prompt.encode("utf-8"),
    stdout=subprocess.PIPE
)
print(result.stdout.decode("utf-8"))

Allison Hill is prescribed Lisinopril.




In [53]:
user_query = "Do you know about Allen Robinson in the dataset?"

full_prompt = llama_preprompt + "\n" + user_query

# Run the LLaMA model with the combined prompt
result = subprocess.run(
    [ollama_path, "run", "llama3.2"],
    input=full_prompt.encode("utf-8"),
    stdout=subprocess.PIPE
)
print(result.stdout.decode("utf-8"))

Yes, I do know about Allen Robinson. According to the dataset, he is a 54-year-old male patient who was treated at the Psychiatry department. His blood pressure reading is not mentioned in his profile, but he does report having a headache. That's the information available about him from the dataset.




## Huggingface 

Om Llama te gebruiken via Huggingface moet er toegang geven worden aan je huggingface account. Daarom moet er ingelogd worden op huggingface

In [54]:
load_dotenv("HF_TOKEN.env")
hf_token = os.getenv("HF_TOKEN")

### Model ophalen

Het model wordt opgehaald en gedownload

In [53]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B", token=hf_token)
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B", token=hf_token)

model.safetensors:  10%|#         | 252M/2.47G [00:00<?, ?B/s]

KeyboardInterrupt: 

### Data transformeren voor trainen

In [17]:
dataset = load_dataset("json", data_files={"train": "mock_patient_data.jsonl"})
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 100
    })
})


In [19]:
print(dataset.column_names)
print(dataset['train'][0])  # Bekijk de eerste rij


{'train': ['instruction', 'input', 'output']}
{'instruction': "Summarize the patient's medical profile.", 'input': 'Name: Allison Hill, Age: 82, Gender: Male, Blood Pressure: 118/68, Medications: Lisinopril, Department: Psychiatry', 'output': 'Allison Hill is a 82-year-old male treated in the Psychiatry department. The patient has a blood pressure reading of 118/68 and is prescribed Lisinopril.'}


### Tokenizen

De tekst in iedere row wordt vervangen door cijfers (tokens) die een stuk tekst representeren. 

In [48]:
def preprocess_function(example):
    """
    Preprocess a single example into the format expected by the model.
    """
    try:
        # Combine instruction and input
        input_text = example["instruction"] + " " + example["input"]
        # Tokenize input
        model_inputs = tokenizer(input_text, padding="max_length", truncation=True, max_length=512)

        # Tokenize output (labels)
        labels = tokenizer(example["output"], padding="max_length", truncation=True, max_length=512).input_ids
        model_inputs["labels"] = labels

        return model_inputs
    except Exception as e:
        print("Error processing:", example)
        print("Error message:", e)
        return {}


In [51]:
tokenized_datasets = dataset["train"].map(preprocess_function, batched=False)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [52]:
print(tokenized_datasets)
print(tokenized_datasets[0])


Dataset({
    features: ['instruction', 'input', 'output', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 100
})
{'instruction': "Summarize the patient's medical profile.", 'input': 'Name: Allison Hill, Age: 82, Gender: Male, Blood Pressure: 118/68, Medications: Lisinopril, Department: Psychiatry', 'output': 'Allison Hill is a 82-year-old male treated in the Psychiatry department. The patient has a blood pressure reading of 118/68 and is prescribed Lisinopril.', 'input_ids': [128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004

### Llama Trainen

Na het tokenizen wordt Llama getraind

In [10]:
training_args = TrainingArguments(
    output_dir="./llama_finetuned",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    max_steps=60,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=2,
    fp16=True,  # Mixed precision voor snelheid
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    tokenizer=tokenizer,
)

trainer.train()

### Model opslaan

In [56]:
trainer.save_model("./llama_finetuned")
tokenizer.save_pretrained("/content/llama_finetuned")

### Model testen

In [ ]:
patient_input = "Naam: John Doe, Leeftijd: 45, Geslacht: Man, Medicatie: Lisinopril, OpnameDatum: 2024-04-10, Afdeling: Cardiologie"

# Tokeniseer de invoer
inputs = tokenizer(patient_input, return_tensors="pt", padding=True, truncation=True, max_length=512)

# Voorspel de output
outputs = model.generate(inputs["input_ids"], max_length=150)

# Decodeer de output
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(f"Voorspelde diagnose: {generated_text}")
